In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import * 
from pyspark.sql.window import Window

In [0]:
df=spark.read.format("parquet").load("abfss://bronze@databricksstorageak.dfs.core.windows.net/customers")

In [0]:
df=df.drop(col("_rescued_data"))

In [0]:
df=df.withColumn("domains",split(col('email'),'@')[1])


In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers",ascending=False)

In [0]:
df_gmail=df.filter(col('domains')=='gmail.com')
df_gmail.display()
df_yahoo=df.filter(col('domains')=='yahoo.com')
df_yahoo.display()


In [0]:
df=df.withColumn("full_name",concat(col('first_name'),lit(' '),col('last_name')))


In [0]:
df=df.drop("first_name","last_name")

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databricksstorageak.dfs.core.windows.net/customers")

In [0]:
%sql
create schema if not exists catabricks_cat.silver

In [0]:
%sql
create table if not exists catabricks_cat.silver.customers_silver using delta 
location 'abfss://silver@databricksstorageak.dfs.core.windows.net/customers'